In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time


In [31]:
url_data ="https://admin.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E{}&cat=Assistances&agg=PerGame"
years = list(range(2001, 2022))

In [32]:
for year in years:
    with open("assists_leaders_htmls/{}.html".format(year), "w+" , encoding="utf-8") as file :
        url = url_data.format(year)
        data = requests.get(url)
        file.write(data.text)
        

In [35]:
dfs = []
for year in years:
    with open("assists_leaders_htmls/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed")
    player_table = soup.find_all(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_ctl01_gvResults")[0]
    assists = pd.read_html(str(player_table))[0]
    assists["Year"] = year
    dfs.append(assists)

In [36]:
assists = pd.concat(dfs)
assists

,Rank,Player,Team,GamesG,AssistsAS,Ave.,Per 40 min.40m,Year
0,1,"BENNETT, ELMERBENNETT, E.",BKNTau CeramicaTau Ceramica,15,79,5.27,6.96,2001
1,2,"HAWKINS, MICHAELHAWKINS, M.",WKSIdea SlaskIdea Slask,14,70,5.00,6.26,2001
2,3,"CELESTAND, JOHNCELESTAND, J.","2 FORSkipper BolognaSkipper Bologna, ASVAsvel ...",13,62,4.77,7.90,2001
3,4,"PHELPS, DERRICKPHELPS, D.",BERAlba BerlinAlba Berlin,13,53,4.08,5.02,2001
4,5,"PAPALOUKAS, THEODOROSPAPALOUKAS, T.",OLYOlympiacosOlympiacos,19,76,4.00,5.94,2001
...,...,...,...,...,...,...,...,...
37,38,"HACKETT, DANIELHACKETT, D.",CSKCSKA MoscowCSKA Moscow,22,57,2.59,4.81,2021
38,39,"SIMON, KRUNOSLAVSIMON, K.",EFSAnadolu Efes IstanbulAnadolu Efes Istanbul,27,69,2.56,4.83,2021
39,40,"PAPAPETROU, IOANNISPAPAPETROU, I.",PAOPanathinaikos OPAP AthensPanathinaikos OPAP...,23,58,2.52,3.47,2021
40,41,"WALDEN, COREYWALDEN, C.",BAYFC Bayern MunichFC Bayern Munich,25,63,2.52,4.50,2021


In [25]:
engine = create_engine('sqlite:///assists_leaders.db')
Session = sessionmaker(bind=engine)
session = Session()

In [26]:
assists.to_sql(
    name = "assists_leaders",
    con = engine,
    index = True,
    if_exists = "replace"
)

666

In [27]:
assists_select = session.execute('select * from "assists_leaders"')
assists_select.all()

[(0, 1, 'BENNETT, ELMERBENNETT, E.', 'BKNTau CeramicaTau Ceramica', 15, 79, 5.27, 6.96),
 (1, 2, 'HAWKINS, MICHAELHAWKINS, M.', 'WKSIdea SlaskIdea Slask', 14, 70, 5.0, 6.26),
 (2, 3, 'CELESTAND, JOHNCELESTAND, J.', '2 FORSkipper BolognaSkipper Bologna, ASVAsvel BasketAsvel Basket', 13, 62, 4.77, 7.9),
 (3, 4, 'PHELPS, DERRICKPHELPS, D.', 'BERAlba BerlinAlba Berlin', 13, 53, 4.08, 5.02),
 (4, 5, 'PAPALOUKAS, THEODOROSPAPALOUKAS, T.', 'OLYOlympiacosOlympiacos', 19, 76, 4.0, 5.94),
 (5, 6, 'McDONALD, ARRIELMCDONALD, A.', 'MACMaccabi EliteMaccabi Elite', 20, 76, 3.8, 4.78),
 (6, 7, 'EDNEY, TYUSEDNEY, T.', 'TREBenetton BasketBenetton Basket', 19, 72, 3.79, 5.0),
 (7, 8, 'LAKOVIC, JAKALAKOVIC, J.', 'MESKRKA Novo MestoKRKA Novo Mesto', 14, 53, 3.79, 4.66),
 (8, 9, 'BOOKER, MELVINBOOKER, M.', 'PESScavolini PesaroScavolini Pesaro', 20, 75, 3.75, 4.44),
 (9, 10, 'LUKOVSKI, DRAGANLUKOVSKI, D.', 'PAUPau-OrthezPau-Orthez', 14, 50, 3.57, 5.7),
 (10, 11, 'MC CANTS, CURTISMC CANTS, C.', 'CSKCSKA Mosco